In [ ]:
# default_exp docments

# Docments

> Document parameters using comments.

In [ ]:
#export
from tokenize import tokenize,COMMENT
from ast import parse,FunctionDef
from io import BytesIO
from textwrap import dedent
from fastcore.basics import *
from types import SimpleNamespace
from inspect import getsource,isfunction,isclass,signature,Parameter

import re

In [ ]:
#hide
from nbdev.showdoc import *

`docments` provides programmatic access to comments in function parameters and return types. It can be used to create more developer-friendly documentation, CLI, etc tools.

## Why?

Without docments, if you want to document your parameters, you have to repeat param names in docstrings, since they're already in the function signature. The parameters have to be kept synchronized in the two places as you change your code. Readers of your code have to look back and forth between two places to understand what's happening. So it's more work for you, and for your users.

Furthermore, to have parameter documentation formatted nicely without docments, you have to use special magic docstring formatting, often with [odd quirks](https://stackoverflow.com/questions/62167540/why-do-definitions-have-a-space-before-the-colon-in-numpy-docstring-sections), which is a pain to create and maintain, and awkward to read in code. For instance, using [numpy-style documentation](https://numpydoc.readthedocs.io/en/latest/format.html):

In [ ]:
def add(a:int, b:int=0)->int:
    """The sum of two numbers.

Parameters
----------
x : int
    the 1st number to add
y : int
    the 2nd number to add (default: 0)

Returns
-------
int
    the result of adding `a` to `b`"""
    return a+b

By comparison, here's the same thing using docments:

In [ ]:
def add(
    a:int, # the 1st number to add
    b=0,   # the 2nd number to add
)->int:    # the result of adding `a` to `b`
    "The sum of two numbers."
    return a+b

## Usage

In [ ]:
#export
def _parses(s):
    "Parse Python code in string or function object `s`"
    return parse(dedent(getsource(s) if isfunction(s) else s))

def _tokens(s):
    "Tokenize Python code in string or function object `s`"
    if isfunction(s): s = getsource(s)
    return tokenize(BytesIO(s.encode('utf-8')).readline)

_clean_re = re.compile('^\s*#(.*)\s*$')
def _clean_comment(s):
    res = _clean_re.findall(s)
    return res[0] if res else None

def _param_locs(s, returns=True):
    "`dict` of parameter line numbers to names"
    body = _parses(s).body
    if len(body)!=1or not isinstance(body[0], FunctionDef): return None
    defn = body[0]
    res = {arg.lineno:arg.arg for arg in defn.args.args}
    if returns and defn.returns: res[defn.returns.lineno] = 'return'
    return res

In [ ]:
#export
def _get_comment(line, arg, comments, parms):
    if line in comments: return comments[line].strip()
    line -= 1
    res = []
    while line and line in comments and line not in parms:
        res.append(comments[line])
        line -= 1
    return dedent('\n'.join(reversed(res))) if res else None

def _get_full(anno, name, default, docs):
    if anno==Parameter.empty and default!=Parameter.empty: anno = type(default)
    return AttrDict(docment=docs.get(name), anno=anno, default=default)

In [ ]:
#export
def docments(s, full=False, returns=True):
    "`dict` of parameter names to 'docment-style' comments in function or string `s`"
    if isclass(s): s = s.__init__ # Constructor for a class
    comments = {o.start[0]:_clean_comment(o.string) for o in _tokens(s) if o.type==COMMENT}
    parms = _param_locs(s, returns=returns)
    docs = {arg:_get_comment(line, arg, comments, parms) for line,arg in parms.items()}
    if not full: return docs

    if isinstance(s,str): s = eval(s)
    sig = signature(s)
    res = {arg:_get_full(p.annotation, p.name, p.default, docs) for arg,p in sig.parameters.items()}
    if returns: res['return'] = _get_full(sig.return_annotation, 'return', Parameter.empty, docs)
    return res

The returned `dict` has parameter names as keys, and comments as values. The return value comment appears in the `return`, unless `returns=False`. Using the `add` definition above, we get:

In [ ]:
docments(add)

{'a': 'the 1st number to add',
 'b': 'the 2nd number to add',
 'return': 'the result of adding `a` to `b`'}

This is the same format that `__annotations__` uses in Python.

In [ ]:
add.__annotations__

{'a': int, 'return': int}

Pass `full=True` to get the annotation and default value for each parameter, along with its docment. Note that the type annotation is inferred from the default value, if the annotation is empty and a default is supplied.

In [ ]:
docments(add, full=True)

{'a': - docment: the 1st number to add
 - anno: <class 'int'>
 - default: <class 'inspect._empty'>,
 'b': - docment: the 2nd number to add
 - anno: <class 'int'>
 - default: 0,
 'return': - docment: the result of adding `a` to `b`
 - anno: <class 'int'>
 - default: <class 'inspect._empty'>}

If you need more space document a parameter, place one or more lines of comments above the parameter, or above the return type. You can mix-and-match these docment styles:

In [ ]:
def add(
    # The first operand
    a:int,
    # This is the second of the operands to the *addition* operator.
    # Note that passing a negative value here is the equivalent of the *subtraction* operator.
    b:int,
)->int: # The result is calculated using Python's builtin `+` operator.
    "Add `a` to `b`"
    return a+b

In [ ]:
docments(add)

{'a': 'The first operand',
 'b': 'This is the second of the operands to the *addition* operator.\nNote that passing a negative value here is the equivalent of the *subtraction* operator.',
 'return': "The result is calculated using Python's builtin `+` operator."}

You can also use documents with classes and methods:

In [ ]:
class Adder:
    "An addition calculator"
    def __init__(self,
        a:int, # First operand
        b:int, # 2nd operand
    ): self.a,self.b = a,b
    
    def calculate(self
                 )->int: # Integral result of addition operator
        "Add `a` to `b`"
        return a+b

In [ ]:
docments(Adder),docments(Adder.calculate)

({'self': None, 'a': 'First operand', 'b': '2nd operand'},
 {'self': None, 'return': 'Integral result of addition operator'})

## Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_test.ipynb.
Converted 01_basics.ipynb.
Converted 02_foundation.ipynb.
Converted 03_xtras.ipynb.
Converted 03a_parallel.ipynb.
Converted 03b_net.ipynb.
Converted 04_dispatch.ipynb.
Converted 05_transform.ipynb.
Converted 06_docments.ipynb.
Converted 07_meta.ipynb.
Converted 08_script.ipynb.
Converted index.ipynb.
Converted parallel_win.ipynb.
